In [8]:
import requests
import string
from pprint import pprint
import re
import os
import tempfile
from bs4 import BeautifulSoup
from collections import Counter, OrderedDict
from operator import itemgetter
import numpy as np
hookRegex = re.compile(r'\[.*\]')
tireRegex = re.compile(r' -')
chars_to_remove = string.punctuation.replace('-', '') + '—„“»«'
print(chars_to_remove)
api_url = 'http://api.genius.com'
base_url = 'http://genius.com'
headers = {'Authorization': 'Bearer ' + open('./.bearer_token.txt', 'r').read().rstrip('\n')}

!"#$%&'()*+,./:;<=>?@[\]^_`{|}~—„“»«


In [9]:
def remove_unnecessary(textfile):
    return tireRegex.sub('', hookRegex.sub('',textfile).translate(str.maketrans('','',chars_to_remove)))

class Artist(object):
    
    def __init__(self, name):
        self.name = name
        
    artist_id = ''
    songlist = []
    
    def get_artist_id(self):
        print("Finding the ID of an artist " + self.name)
        search_url = api_url + '/search'
        data = {'q': self.name}
        response = requests.get(search_url, params=data, headers=headers)
        self.artist_id = response.json()['response']['hits'][0]['result']['primary_artist']['api_path']
        print("ID of an artist " + self.name + " is " + self.artist_id[9:])
        with open('./artists_id_list.txt', 'r+') as textfile:
            if self.artist_id in textfile.read():
                pass
            else:
                textfile.write(textfile.read() + '\n' + self.name + "  -  " + self.artist_id)
        return
    #artists/143013/songs?sort=popularity&per_page=50
    #artists/143013/songs?sort=popularity&per_page=50&page=3
    def get_songlist(self):
        list_of_songs = []
        print("Getting the list of songs")
        for page in range(1,4):
            res = requests.get(api_url + self.artist_id + '/songs?sort=popularity&per_page=50&page=' + str(page), headers=headers)
            print("Going through page " + str(page))
            for song in res.json()['response']['songs']:
#                 print(song['full_title'])
                if ("Ft." in song['full_title']) | ("Remix" in song['full_title']):
                    print(song['full_title'] + " is a feat, skipping")
                else:
                    song_url = song['url']
                    list_of_songs.append(song_url)
        self.songlist = list_of_songs
        return
    
    
    def write_lyrics_from_path(self):
        with open("./lyrics/" + self.name.replace(' ', '_') + '_lyrics.txt', 'w') as lyricsfile:
            for song in self.songlist:
                print(str(self.songlist.index(song)) + " Now scraping " + song[19:])
                page_url = song
                res = requests.get(page_url, headers=headers)
                res.text
                page = requests.get(page_url)
                html = BeautifulSoup(page.text, 'html.parser')
                lyrics = html.find(class_='lyrics').get_text()
                lyricsfile.write(remove_unnecessary(lyrics.lower()))
        return
    
    def perform_task(self, write_id=False):
        self.get_artist_id()
        self.get_songlist()
        self.write_lyrics_from_path()
        return


In [10]:
%%time
with open('./artists_source.txt', 'r') as id_list:
    for artist in id_list.readlines():
        the_artist = Artist(artist.rstrip("\n"))
        the_artist.perform_task(write_id=True)

Finding the ID of an artist oxxxymiron
ID of an artist oxxxymiron is 24422
Getting the list of songs
Going through page 1
12/04/15: Oxxxymiron vs. Johnyboy by Versus Battle (Ft. Johnyboy & Oxxxymiron) is a feat, skipping
Fata Morgana by Markul (Ft. Oxxxymiron) is a feat, skipping
Безумие (Madness) [Remix] by ЛСП (LSP) (Ft. Oxxxymiron) is a feat, skipping
Пора возвращаться домой (PVD) by БИ-2 (BI-2) (Ft. Oxxxymiron) is a feat, skipping
13/08/2017: Oxxxymiron vs. Слава КПСС by Versus Battle (Ft. Oxxxymiron & Слава КПСС (Slava KPSS)) is a feat, skipping
Больше Бена (Bolshe Bena) by Oxxxymiron (Ft. Охра (Ohra)) is a feat, skipping
Darkside by Oxxxymiron (Ft. MadChild) is a feat, skipping
Imperial by Porchy (Ft. ЛСП (LSP) & Oxxxymiron) is a feat, skipping
Бездыханным (Breathless) by Жак-Энтони (Jacques-Anthony) (Ft. Oxxxymiron) is a feat, skipping
Машина Прогресса (Progress Machine) by Ка-тет (Ka-tet) (Ft. Oxxxymiron) is a feat, skipping
Привет со дна  (Hello from the bottom) by Oxxxymiron 

Going through page 2
Going through page 3
0 Now scraping Makulatura-reprise-lyrics
1 Now scraping Makulatura-nate-diaz-lyrics
2 Now scraping Makulatura-how-the-dead-love-lyrics
3 Now scraping Makulatura-in-a-fire-lyrics
4 Now scraping Makulatura-singularity-lyrics
5 Now scraping Makulatura-30-30-let-kak-myortv-lyrics
6 Now scraping Makulatura-the-smartest-people-alive-lyrics
7 Now scraping Makulatura-limb-lyrics
8 Now scraping Makulatura-wrists-lyrics
9 Now scraping Makulatura-together-annotated
10 Now scraping Makulatura-30-dead-for-30-years-lyrics
11 Now scraping Makulatura-letuchiy-gollandec-lyrics
12 Now scraping Makulatura-nojevoe-lyrics
13 Now scraping Makulatura-alzheimer-lyrics
14 Now scraping Makulatura-gift-horse-lyrics
15 Now scraping Makulatura-samiy-otstoyniy-den-lyrics
16 Now scraping Makulatura-the-all-universe-lyrics
17 Now scraping Makulatura-beach-lyrics
18 Now scraping Makulatura-walter-lyrics
19 Now scraping Makulatura-stage-lyrics
20 Now scraping Makulatura-the-fat

62 Now scraping Guf-naugad-lyrics
63 Now scraping Centr-porosya-lyrics
64 Now scraping Guf-metropolitan-mail-lyrics
65 Now scraping Guf-dlya-nee-lyrics
66 Now scraping Guf-dno-lyrics
67 Now scraping Guf-and-rigos-long-way-lyrics
68 Now scraping Centr-po-zhesti-lyrics
69 Now scraping Guf-and-rigos-inhale-lyrics
70 Now scraping Guf-excursion-lyrics
71 Now scraping Centr-77-zhara-77-lyrics
72 Now scraping Guf-and-rigos-parasite-lyrics
73 Now scraping Guf-ka-kalpak-lyrics
74 Now scraping Guf-and-rigos-renegade-lyrics
75 Now scraping Guf-novenkiy-lyrics
76 Now scraping Guf-and-rigos-low-profile-lyrics
77 Now scraping Guf-and-rigos-bezvilazno-lyrics
78 Now scraping Centr-peremotka-lyrics
79 Now scraping Guf-sam-skit-lyrics
80 Now scraping Guf-skit-ot-sanka-lyrics
81 Now scraping Centr-slaydi-lyrics
82 Now scraping Centr-kak-nastroenie-lyrics
83 Now scraping Guf-hobby-lyrics
84 Now scraping Guf-and-rigos-industry-lyrics
85 Now scraping Guf-and-rigos-pali-jk-lyrics
86 Now scraping Centr-sosedi

1 Now scraping Lsp-crawl-lyrics
2 Now scraping Lsp-mld-lyrics
3 Now scraping Lsp-cancan-lyrics
4 Now scraping Lsp-numbers-lyrics
5 Now scraping Lsp-white-dance-lyrics
6 Now scraping Lsp-the-little-prince-lyrics
7 Now scraping Lsp-and-pharaoh-neon-lyrics
8 Now scraping Lsp-one-more-day-lyrics
9 Now scraping Lsp-and-pharaoh-pornstar-lyrics
10 Now scraping Lsp-labyrinth-of-reflections-lyrics
11 Now scraping Lsp-spit-into-eternity-lyrics
12 Now scraping Lsp-moneys-not-a-problem-lyrics
13 Now scraping Lsp-end-of-the-world-lyrics
14 Now scraping Lsp-garbage-lyrics
15 Now scraping Lsp-ok-lyrics
16 Now scraping Lsp-and-pharaoh-goldfish-lyrics
17 Now scraping Lsp-resurrection-lyrics
18 Now scraping Lsp-poteryan-i-ne-nayden-lyrics
19 Now scraping Lsp-and-pharaoh-pit-lyrics
20 Now scraping Lsp-and-pharaoh-cake-lyrics
21 Now scraping Lsp-blue-lyrics
22 Now scraping Lsp-cocktail-lyrics
23 Now scraping Lsp-bitch-lyrics
24 Now scraping Lsp-levels-lyrics
25 Now scraping Lsp-vinaigrette-lyrics
26 Now s

1 Now scraping Scriptonite-situation-lyrics
2 Now scraping Scriptonite-animals-lyrics
3 Now scraping Scriptonite-for-real-lyrics
4 Now scraping Scriptonite-butter-lyrics
5 Now scraping Scriptonite-napomni-lyrics
6 Now scraping Scriptonite-intro-time-to-come-back-lyrics
7 Now scraping Scriptonite-priton-lyrics
8 Now scraping Scriptonite-ostav-ih-lyrics
9 Now scraping Scriptonite-dont-need-to-recognize-me-lyrics
10 Now scraping Scriptonite-mirrors-lyrics
11 Now scraping Scriptonite-718-mr-718-lyrics
12 Now scraping Scriptonite-1000-lyrics
13 Now scraping Scriptonite-temno-lyrics
14 Now scraping Scriptonite-waste-of-time-lyrics
15 Now scraping 104-and-truwer-safari-lyrics
16 Now scraping Scriptonite-it-is-love-lyrics
17 Now scraping Scriptonite-party-lyrics
18 Now scraping Scriptonite-tantsuy-sama-lyrics
19 Now scraping Scriptonite-bro-lyrics
20 Now scraping Scriptonite-lokony-lyrics
21 Now scraping Scriptonite-boy-lyrics
22 Now scraping Scriptonite-interview-lyrics
23 Now scraping Script

...

Going through page 2
Наше лето (Nashe leto) by Баста (Basta) (Ft. МакSим (MakSim)) is a feat, skipping
Виски и кокаин (Viski i kokain) by Баста (Basta) (Ft. Нервы (Nervy) & Смоки Мо (Smoky Mo)) is a feat, skipping
Через тернии к звездам (Through hardship...) by NEL (Ft. Баста (Basta)) is a feat, skipping
Обернись (Turn around) by Баста (Basta) (Ft. Город 312 (Gorod 312)) is a feat, skipping
Каменные цветы (Stone flowers ) by Баста (Basta) & Смоки Мо (Smoky Mo) (Ft. Елена Ваенга (Elena Vaenga)) is a feat, skipping
Космос (Cosmos) by Баста (Basta) (Ft. Чаруша (Charusha)) is a feat, skipping
Ум (Um) by Баста (Basta) (Ft. UM) is a feat, skipping
Я поднимаюсь над землей (Ya podnimayus' nad zemley) by Баста (Basta) (Ft. Алена Омаргалиева (Alena Omargaliyeva)) is a feat, skipping
Один (Odin) by Баста (Basta) (Ft. Смоки Мо (Smoky Mo)) is a feat, skipping
С надеждой на крылья (With hope for wings) by Баста (Basta) (Ft. Нервы (Nervy)) is a feat, skipping
Финальный матч (Final'nyy match) by Баста

Going through page 2
Трафик (Traffic) by Centr (Ft. Смоки Мо (Smoky Mo)) is a feat, skipping
Не на экспорт (Ne na export) by Centr (Ft. TAHDEM Foundation) is a feat, skipping
В норме (V norme) by Centr (Ft. Словетский (Slovetskii)) is a feat, skipping
Качели. часть 2 (Kacheli. part 2) by Centr (Ft. Ноггано (Noggano)) is a feat, skipping
Клещи (Kleshi) by Slim (Ft. Каспийский Груз (KG)) is a feat, skipping
Беркуты (Berkuty) by Slim (Ft. Барбитурный (Barbiturny) & Расколъ (Raskol 666)) is a feat, skipping
Далеко (Daleko) by Centr (Ft. А-Студио (A’Studio)) is a feat, skipping
Железное Небо (Zheleznoe Nebo) by Centr (Ft. 25/17, Принцип (Princip) & Рома Жиган (Roma Jigan)) is a feat, skipping
Не не (Ne ne) by Каспийский Груз (KG) (Ft. Slim) is a feat, skipping
Двигай (Dvigay) by Slim (Ft. Daffy (Russia)) is a feat, skipping
Ночь (Night) by Centr (Ft. Тати (Tati)) is a feat, skipping
Легенды (Legendi) by Centr (Ft. 5 Плюх (Five P)) is a feat, skipping
Летняя (Summer) by Slim (Ft. Daffy (Russ

1 Now scraping Schokk-htm-lyrics
2 Now scraping Schokk-rubchinsky-sweater-lyrics
3 Now scraping Schokk-round-lyrics
4 Now scraping Schokk-amen-lyrics
5 Now scraping Schokk-misanthrope-lyrics
6 Now scraping Schokk-ponedelnik-lyrics
7 Now scraping Schokk-tak-nado-lyrics
8 Now scraping Schokk-intro-lyrics
9 Now scraping Schokk-tmvt-lyrics
10 Now scraping Schokk-shpana-lyrics
11 Now scraping Schokk-questions-lyrics
12 Now scraping Schokk-bamberg-lyrics
13 Now scraping Schokk-pink-wine-lyrics
14 Now scraping Schokk-your-life-lyrics
15 Now scraping Schokk-toni-montano-lyrics
16 Now scraping Schokk-kandagach-skit-lyrics
17 Now scraping Schokk-benz-lyrics
18 Now scraping Schokk-ii-walk-ii-lyrics
19 Now scraping Schokk-iv-lyrics
20 Now scraping Schokk-rapelectroschokk-ii-lyrics
21 Now scraping Schokk-we-throw-stones-lyrics
22 Now scraping Schokk-intro-sbd-lyrics
23 Now scraping Schokk-lubov-kapitana-lyrics
24 Now scraping Schokk-sad-lyrics
25 Now scraping Schokk-outro-lyrics
26 Now scraping Sch

Going through page 3
23/11/13: Yanix vs. Galat by Versus Battle (Ft. Galat & Yanix) is a feat, skipping
Пей до дна (bottoms up) by Yanix (Ft. Bonus B & GidraGidra) is a feat, skipping
НПНБ (NPNB) by Мезза (Mezza) (Ft. Yanix) is a feat, skipping
Всё и сразу (All At Once) [Remix] by Yanix (Ft. ЛСП (LSP)) is a feat, skipping
Без Тормозов by D.masta (Ft. Booguy & Yanix) is a feat, skipping
Черный пояс (black belt) by Yanix (Ft. Мезза (Mezza)) is a feat, skipping
Еще еще (more and more) by Yanix (Ft. Чаян Фамали (Chayan Famaly)) is a feat, skipping
Супер индиго (super indigo) by Yanix (Ft. Hash Tag) is a feat, skipping
Холодный Жар (cold fever) by Yanix (Ft. Guts Out) is a feat, skipping
Моим Псам (For my Dogs) by Yanix (Ft. Bonus B, D.masta & GidraGidra) is a feat, skipping
Не один (not alone) by Yanix (Ft. Basic Boy) is a feat, skipping
0 Now scraping Yanix-hypin-lyrics
1 Now scraping Yanix-poppin-lyrics
2 Now scraping Yanix-and-thomas-mraz-i-want-to-you-lyrics
3 Now scraping Yanix-and-th

71 Now scraping Anacondaz-more-volnuyetsya-lyrics
Finding the ID of an artist карандаш
ID of an artist карандаш is 664698
Getting the list of songs
Going through page 1
Тату (Tattoo) by Тимати (Timati) (Ft. Джиган (Geegun), Карандаш (Karandash), L’One, Крэк (Rayon) & Варчун (Varchun)) is a feat, skipping
Tissot by Карандаш (Karandash) (Ft. Ёлка (Yolka)) is a feat, skipping
Сделай меня плохим (Make Me Bad) by Карандаш (Karandash) (Ft. Shaman (Russia)) is a feat, skipping
Камеры (Cameras) by Карандаш (Karandash) (Ft. Звонкий (Zvonkiy)) is a feat, skipping
Это нормально (Eto Normal'no) (RMX) by Карандаш (Karandash) (Ft. Anacondaz, Крип-а-Крип (Crip-a-Crip), Дабл (Dabl), Dime (Nonamerz), Fuze, Giga, Lenin, Марсель (Marsel), Нигатив (Nigativ), PLC, Сэт (Sette) & Варчун (Varchun)) is a feat, skipping
Королева (The Queen) by Карандаш (Karandash) (Ft. Звонкий (Zvonkiy)) is a feat, skipping
Рэп как шанс (Rap kak shans) by Баста (Basta) (Ft. Крип-а-Крип (Crip-a-Crip), Jahmal (TGK), Карандаш (Kar

In [4]:
def get_percentage(words, total):
    return round(words*100/total, 2)

def ord_dict(input_dict):
    return OrderedDict(sorted(input_dict.items(), key = itemgetter(1), reverse = True))

class Lyric_file(object): 
    the_text = ''
    the_list = []
    the_counter = {}
    the_total = 0
    def __init__(self, location):
        self.location = location
        self.the_text = open(self.location, 'r').read()
        self.the_list = self.the_text.split()
        self.the_counter = Counter(self.the_list)
        self.the_total = len(self.the_list)
        
    def get_most_common(self, number):
        return self.the_counter.most_common(number)
    
    def get_most_common_percentage(self, number):
        result_list = []
        for word in self.get_most_common(number):
            result_list.append((word[0], get_percentage(word[1], len(self.get_list()))))
        return result_list
    
    def get_percentage_of_word(self, word):
        return get_percentage(self.the_counter[word],len(self.the_list))
    
    def get_vocab_size(self):
        return len(self.the_list)/len(self.the_counter)
    

In [5]:
list_tuple = {}
for artist in os.listdir('./lyrics/'):
    lyric = Lyric_file('./lyrics/' + artist)
    list_tuple[artist] = lyric.get_percentage_of_word("я")

pprint(ord_dict(list_tuple))

OrderedDict([('oxxxymiron_lyrics.txt', 2.53)])


In [6]:
list_tuple = {}
for artist in os.listdir('./lyrics/'):
    lyric = Lyric_file('./lyrics/' + artist)
    list_tuple[artist] = lyric.get_vocab_size()
pprint(OrderedDict(sorted(list_tuple.items(), key = itemgetter(1), reverse = True)))
    

OrderedDict([('oxxxymiron_lyrics.txt', 2.7867465173805495)])


In [7]:
list_of_me = ['я', "мой", "моя", "мои", "меня", "мы", "наш"]
lyricobj = Lyric_file('./lyrics/guf_lyrics.txt')

FileNotFoundError: [Errno 2] No such file or directory: './lyrics/guf_lyrics.txt'